In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn import model_selection
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report
from sklearn import metrics
from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
import numpy as np
from vecstack import stacking
from sklearn.ensemble import AdaBoostClassifier

# Preprocesamiento del dataset

In [ ]:
# importa el dataset
data = pd.read_csv('Dataset_DesempeñoAcademico.csv')

In [ ]:
# reemplaza los valores categóricos del target por una variable dummy
# High pasa a ser 1 y Low y Medium pasan a ser 0
cleanup_nums = {"Class":{"H": 1, "L": 0, "M": 0}}
data.replace(cleanup_nums, inplace=True)

In [ ]:
# lista de columnas numéricas
number_columns = ['raisedhands','VisITedResources','AnnouncementsView','Discussion']

In [ ]:
# quita columnas numéricas del dataset y columna target
data_nominal = data.drop(columns=number_columns)
data_nominal = data_nominal.drop(columns=['Class'])

In [ ]:
# función que genera una lista de listas a partir de un df
# cada lista es una fila del dataframe
def generate_matriz2transform(df2transform):
    a = []
    for i in range(len(df2transform)):
        a.append(list(df2transform.loc[i,:]))
    return a

In [ ]:
# instancia la funcion generate_matriz2transform con el df con categorías nominales
matriz = generate_matriz2transform(data_nominal)

In [ ]:
# función que extrae la lista de valores de los atributos nominales de un df y los guarda en un diccionario
# toma como parámetro el df del cual se extraerán dichos atributos
# output: {'genre': ['F','M'], 'Relation':['Father','Mother'],...}
def get_attributes(nominalDF):
    dict_attributes = dict()
    for c in nominalDF.columns:
        dict_attributes[c]=list(pd.unique(nominalDF[c]))
    return dict_attributes

In [ ]:
# instancia la funcion get_attributes con el df con categorías nominales
# tomando en cuenta todas sus filas y todas las columnas
dict_attributes = get_attributes(data_nominal)

In [ ]:
# función que transforma una matriz con atributos nominales en una matriz con atributos numéricos
# toma como parámetros la matriz a transformar y un diccionario con los atributos y sus posibles valores
def transform_matriz(matriz2transform, attributes2transform):
    transformed_rows = list()
    label_encoders = list()
    # para cada atributo en el diccionario de atributos
    for attribute in attributes2transform.keys():
        # genera un objeto LabelEncoder
        le = preprocessing.LabelEncoder()
        # lo entrena con los valores de ese atributo
        le.fit(attributes2transform[attribute])
        # y lo agrega a la lista de encoders
        label_encoders.append(le)
    # para cada fila de la matriz a transformar
    for i in range(len(matriz2transform)):
        converted = list()
        # para cada l en el rango de 0 a la cantidad de encoders
        for l in range(len(label_encoders)):
            # convierte con el encoder l la palabra en la posición l
            conv = label_encoders[l].transform(matriz2transform[i][l:l+1])[0]
            # agrega la conversión a la lista de convertido
            converted.append(conv)
        # agrega la fila con los valores convertidos a la lista
        transformed_rows.append(converted)
    # devuelve la lista de todas las filas convertidas
    return transformed_rows

In [ ]:
# instancia la función transform_matriz con la matriz para transformar
c = transform_matriz(matriz, dict_attributes)

In [ ]:
# pasa la matriz a un df y le asigna nombre a sus columnas
df = pd.DataFrame(c, columns=data_nominal.columns)

In [ ]:
# función que agrega agrega columnas numéricas de un df a otro
# toma como parámetros el df al cual se le agregan columnas, el df del cual se copian las columnas y una lista
# con los nombres de las columnas a copiar
def aggregate_num_columns(df2agg, df_WithNumColumns, num_columns):
    for c in num_columns:
        df2agg[c] = df_WithNumColumns[c]
    return df2agg

In [ ]:
# instancia la funcion aggregate_num_columns con:
# 1) el df (data frame con atributos nominales convertidos a números),
# 2) el data (data frame desde donde extraer el columnas numéricas),
# 3) la lista number_columns que contiene los nombres de las columnas numéricas
df = aggregate_num_columns(df, data, number_columns)

In [ ]:
# vector con el target del dataset
y = data['Class']

# Separación del dataset en desarrollo y held-out

In [ ]:
# DESARROLLO: train_X, train_y   --->   queda dividirlo en training y validation
# HELD-OUT: test_X, test_y
trainX, testX, trainy, testy = train_test_split(df, y,
                                                train_size=0.80,
                                                test_size=0.20,
                                                shuffle=True,
                                                random_state=123,)

# Elección de k-fold para cross-validation

In [ ]:
kfold = model_selection.KFold(n_splits=5, shuffle=True, random_state=123)

### Se define una función que será utilizada en todos los modelos para obtener los valores de TF, FP, TN y FN desde la matriz de confusion

In [ ]:
def metricas(y_true, y_pred):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    print('{0:15} {1}'.format('Accuracy:',round(accuracy,3)))
    print('*********************')
    print('{0:15} {1}'.format('Precision:',round(precision,3)))
    print('{0:15} {1}'.format('Recall:',round(recall,3))) 
    print('{0:15} {1}'.format('F1 score:',round(f1,3)))
    print('*********************')
    print('{0:15} {1}'.format('True positive:',cm[0][0]))
    print('{0:15} {1}'.format('False positive:',cm[0][1]))
    print('{0:15} {1}'.format('False negative:',cm[1][0]))
    print('{0:15} {1}'.format('True negative:',cm[1][1]))

# Entrenamientos

## 1. Bagging con Decision Trees

In [ ]:
cart = DecisionTreeClassifier(max_depth=10)
modelo_dt = BaggingClassifier(base_estimator=cart, n_estimators=40, random_state=0)
modelo_dt.fit(trainX,trainy)
result_dt = model_selection.cross_val_score(modelo_dt, trainX, trainy, cv=kfold)
print('Accuracy Bagging con Decision Trees con 5-fold cross validation en train: %f' % result_dt.mean())

In [ ]:
#modelo_dt.get_params()

In [ ]:
yhat_dt = modelo_dt.predict(testX)
metricas(testy,yhat_dt)

## 2. Boosting con XGBoost

In [ ]:
modelo_xg = XGBClassifier(max_depth=11,
                           min_child_weight=1,
                           learning_rate=0.019,
                           n_estimators=550,
                           silent=True,
                           objective='binary:logistic',
                           gamma=0,
                           max_delta_step=0,
                           subsample=1,
                           colsample_bytree=1,
                           colsample_bylevel=1,
                           reg_alpha=0,
                           reg_lambda=0,
                           scale_pos_weight=30,
                           seed=10,
                           missing=None)
modelo_xg.fit(trainX,trainy)
result_xg = model_selection.cross_val_score(modelo_xg, trainX, trainy, cv=kfold)
print('Accuracy XGBoost con 5-fold cross validation en train: %f' % result_xg.mean())

In [ ]:
#modelo_xg.get_params()

In [ ]:
yhat_xg = modelo_xg.predict(testX)
metricas(testy,yhat_xg)

## 3. Stacking con Decision Tree y XGBoost

Stacking con Decision Tree y XGBoost utilizando los mismos parámetros que en los entrenamientos 1 y 2.

In [ ]:
models = [
    RandomForestClassifier(n_estimators=200,
                            max_depth=10, 
                            n_jobs=1,
                            random_state=0),
    XGBClassifier(random_state=0,
                         n_jobs=1, 
                         learning_rate=0.019, 
                         n_estimators=300, 
                         seed = 5,
                         max_depth=20)
]

In [ ]:
S_train, S_test = stacking(models, trainX, trainy, testX,
                           regression=False,      
                           mode='oof_pred_bag',            
                           metric=accuracy_score,     
                           n_folds=5,           
                           shuffle=True,              
                           random_state=0,
                           verbose=2)

In [ ]:
ada_base = DecisionTreeClassifier(max_depth=10)

In [ ]:
stacking_model = AdaBoostClassifier(base_estimator=ada_base,
                                    n_estimators=500,
                                    random_state=0)
stacking_model.fit(S_train, trainy)

In [ ]:
result_stack = model_selection.cross_val_score(stacking_model, S_train, trainy, cv=kfold)
print('Accuracy Stacking con 5-fold cross validation en train: %f' % result_stack.mean())

In [ ]:
stacking_model.get_params()

In [ ]:
yhat_stack = stacking_model.predict(S_test)
metricas(testy,yhat_stack)

## Gráficos

In [ ]:
import matplotlib.pyplot as plt
 
barWidth = 0.9
bars1 = [0.849, 0.807, 0.859] #métricas de cross-validation con fold=5: orden: Bagging, Boosting, Stacking
bars2 = [0.896, 0.927, 0.885] #métricas de held-out: orden: Bagging, Boosting, Stacking
bars4 = bars1 + bars2
 
r1 = [1,3,5]
r2 = [2,4,6]
r4 = r1 + r2
 
plt.figure(figsize=(10,7))
plt.bar(r1, bars1, width = barWidth, color = (0.3,0.1,0.4,0.6), label='5-fold Accuracy')
plt.bar(r2, bars2, width = barWidth, color = (0.3,0.5,0.4,0.6), label='Held-out Accuracy')
 
plt.legend(loc=4)
 
plt.xticks([r + barWidth for r in range(len(r4))], 
           ['Bagging','Bagging','Boosting','Boosting','Stacking','Stacking'], rotation=90)
 
label = ['n = 0.849', 'n = 0.896', 'n = 0.807', 'n = 0.927', 'n = 0.851', 'n = 0.875']
 
for i in range(len(r4)):
    plt.text(x = r4[i]-0.3 , y = bars4[i] + 0.02, s = label[i], size = 10)

plt.subplots_adjust(bottom= 0.2, top = 1)
 
plt.show()